In [ ]:
import uproot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import sys

# plt.style.use("belle2")

In [ ]:
plt.rcParams.update({
    "axes.labelsize": 16,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 14,
    "figure.titlesize": 18
})

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/07-Python_Functions/")

# Prep-Work

### Import Data

In [ ]:
# import uproot
# import pandas as pd

# # Define your samples (same logic, no need for *_Ds* in filename)
# samples = ["Signal","ccbar"]

# DataFrames = {}  # define empty dictionary to hold dataframes
# Date = "0607"
# Attempt = "0"

# # Loop through all samples and load both trees
# for s in samples:
#     # Base file path
#     if s == "Signal":
#         filepath = "/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root"
#     else:
#         filepath = f"/group/belle2/users2022/amubarak/02-Grid/Sample_KEKCC/Ds2D0e-Generic_Ds_{Date}25_{Attempt}_{s}.root"

#     # Load Dstree normally
#     DataFrames[s] = uproot.concatenate(f"{filepath}:Dstree", library='pd')

#     # Load Ds*tree under a different key
#     DataFrames[f"{s}_Ds*"] = uproot.concatenate(f"{filepath}:Ds*tree", library='pd')

# # # Create combined backgrounds
# # background_samples = ["ccbar", "charged", "ddbar", "mixed", "ssbar", "uubar"]
# # background_samples_Dstar = [f"{s}_Ds*" for s in background_samples]

# # # Concatenate all backgrounds
# # DataFrames["All"] = pd.concat([DataFrames[s] for s in background_samples], ignore_index=True)
# # DataFrames["All_Ds*"] = pd.concat([DataFrames[s] for s in background_samples_Dstar], ignore_index=True)

# # # Combine light-quark backgrounds
# # DataFrames["uds"] = pd.concat([DataFrames[s] for s in ["uubar", "ddbar", "ssbar"]], ignore_index=True)
# # DataFrames["uds_Ds*"] = pd.concat([DataFrames[s] for s in ["uubar_Ds*", "ddbar_Ds*", "ssbar_Ds*"]], ignore_index=True)

In [ ]:
import uproot
import pandas as pd

# === Load only selected branches for correct charge ===
with open("/home/belle2/amubarak/Ds2D0enue_Analysis/03-Grid/Save_var.txt") as f:
    variables_to_load = [
        line.strip().strip(",").strip('"').strip("'")
        for line in f
        if line.strip() and not line.strip().startswith("#")
    ]

# Define your samples (same logic, no need for *_Ds* in filename)
samples = ["Signal", "BB", "ccbar", "ddbar", "ssbar", "taupair", "uubar"]

DataFrames = {}  # define empty dictionary to hold dataframes
Date = "0530"
Attempt = "0"

# Loop through all samples and load both trees
for s in samples:
    # Base file path
    if s == "Signal":
        filepath = "/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root"
    else:
        filepath = f"/group/belle2/users2022/amubarak/02-Grid/Sample_Grid/Ds2D0e-Generic_Ds_{Date}25_{Attempt}_{s}.root"

    # Load Dstree normally
    DataFrames[s] = uproot.concatenate(f"{filepath}:Dstree", 
                                       filter_name=variables_to_load,
                                       library='pd')

    # Load Ds*tree under a different key
    DataFrames[f"{s}_Ds*"] = uproot.concatenate(f"{filepath}:Ds*tree", library='pd')

# Create combined backgrounds
background_samples = ["BB", "ccbar", "ddbar", "ssbar", "taupair", "uubar"]
background_samples_Dstar = [f"{s}_Ds*" for s in background_samples]

# Concatenate all backgrounds
DataFrames["All"] = pd.concat([DataFrames[s] for s in background_samples], ignore_index=True)
DataFrames["All_Ds*"] = pd.concat([DataFrames[s] for s in background_samples_Dstar], ignore_index=True)

# Combine light-quark backgrounds
DataFrames["uds"] = pd.concat([DataFrames[s] for s in ["uubar", "ddbar", "ssbar"]], ignore_index=True)
DataFrames["uds_Ds*"] = pd.concat([DataFrames[s] for s in ["uubar_Ds*", "ddbar_Ds*", "ssbar_Ds*"]], ignore_index=True)

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

The line below is to look at the available variables.

In [ ]:
DataFrames["ccbar_Ds*"].columns.tolist()

In [ ]:
DataFrames["Signal_Ds*"].columns.tolist()

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# # Electron ID
# #-------------------
# DataFrames["Signal_Ds*"] = DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=0.5) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=0.5)]
# DataFrames["All_Ds*"] = DataFrames["All_Ds*"][(DataFrames["All_Ds*"]['gamma_beamBackgroundSuppressionScore']>=0.5) & (DataFrames["All_Ds*"]['gamma_fakePhotonSuppressionScore']>=0.5)]

# Photon Conversion
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["All"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb_rank']==1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb_rank']==1]

# D0 Invariant Mass
#-----------------------
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_D0_sideband']==1)]
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['Ds_D0_sideband']==1)]

# KaonID
#-----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['K_kaonID']>=0.5]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['K_kaonID']>=0.5]

In [ ]:
print(abs(DataFrames["Signal_Ds*"]['gamma_genMotherPDG']).value_counts(normalize=True,dropna=False).head(10))
print(abs(DataFrames["ccbar_Ds*"]['gamma_genMotherPDG']).value_counts(normalize=True,dropna=False).head(10))

In [ ]:
print(abs(DataFrames["ccbar_Ds*"][(abs(DataFrames["ccbar_Ds*"]["gamma_genMotherPDG"])==423)]['Ds_c_mcPDG']).value_counts(normalize=True,dropna=False).head(10))

In [ ]:
print(abs(DataFrames["Signal_Ds*"]['Ds_star_mcPDG']).value_counts(normalize=True,dropna=False).head(10))
print(abs(DataFrames["All_Ds*"]['Ds_star_mcPDG']).value_counts(normalize=True,dropna=False).head(10))

# Candidates

In [ ]:
# Stacked = False
# Density = False
# Bins = 30
# Range = [0, 30]
# Hits = -1
# Op = 0.58
# RR = 0.4
# BS = 0.5
# FS = 0.5
# var = '__ncandidates__'
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$isSignal(D_s^{*+})=1$'

# labels=[label1]
# colors=['#7eb0d5']

# data = [DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)][var],
#        ]


# plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')

# # Title
# #---------
# plt.title(r'$D_s^{*+} \rightarrow D_s^{+} \gamma$', loc = "left")
# plt.title(r'$\bf Signal\;Events$', loc = "right")
# # plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# # plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# # Label
# #---------
# plt.ylabel(r'$Entries$')
# plt.xlabel(r'$\# \; of \; Candidates \; per \; Event$')
# # plt.yscale("log") 
# # plt.legend()
# plt.show()

# Single Variable  
The code below is to look at any specific variable closely.

In [ ]:
Stacked = True
Density = True
Bins = 50
var = 'Ds_Ds_starminusDs_M_Correction'
Range = [0., 0.4]
BD = -1
# perBin = ((Range[1] - Range[0])/Bins)
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$isSignal(D_s^{+})=1$'
label2= r'$isSignal(D_s^{+})=0$'
label3= r'$NaN$'

labels=[label1,label2,label3]
colors=["#4C6EB1",'#fd7f6f','purple']

data = [DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal']==1)][var],
        DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal']==0)][var],
        DataFrames["Signal"][(DataFrames["Signal"]['Ds_isSignal'].isna())][var]
       ]


plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, linewidth=2, stacked=Stacked, density=Density, bins=Bins, histtype='step')
# plt.axvspan(Range[0],-0.02,color='gray',alpha=0.2)
# plt.axvspan(0.02,Range[1],color='gray',alpha=0.2)
# plt.axvline(-0.02,ls='--',color='gray')
# plt.axvline(0.02,ls='--',color='gray')

# Title
#---------
# Signal
# plt.title(r'$2M\;Events$', loc = "left")
plt.title(r'$\bf Signal\;Events$', loc = "right")
# # Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{*+} - D_s^{+})\;[GeV/c^{2}]$')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# === Config ===
Stacked = True
Density = True
Bins = 50
var = 'Ds_star_massDifference_0'
Range = [0.0, 0.4]
perBin = ((Range[1] - Range[0]) / Bins) * 1000  # MeV
print("Width Per Bin: {:.2f} MeV".format(perBin))

# === Labels & Colors ===
label1 = r'$isSignal(D_s^{*+})=1$'
label2 = r'$isSignal(D_s^{*+})=0$'
label3 = r'$NaN$'
labels = [label1, label2, label3]
colors = ["#4C6EB1", "#fd7f6f", "purple"]

# === Data Selection ===
df = DataFrames["Signal_Ds*"]
data = [
    df[df['Ds_star_isSignal'] == 1][var],
    df[df['Ds_star_isSignal'] == 0][var],
    df[df['Ds_star_isSignal'].isna()][var]
]

# === Plot ===
plt.hist(data[::-1], color=colors[::-1], label=labels[::-1],
         alpha=1, range=Range, linewidth=2,
         stacked=Stacked, density=Density,
         bins=Bins, histtype='step')

# Titles
# plt.title(r'$2M\;Events$', loc="left")
plt.title(r'$\bf Signal\;Events$', loc="right")

# Axis Labels
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{*+} - D_s^{+})\;[\mathrm{GeV}/c^{2}]$')

# Legend
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = True
Bins = 50
Range = [0, 1]
Op = -1
dM = -1
i = 'gamma_beamBackgroundSuppressionScore'
perBin = ((Range[1] - Range[0])/Bins)*1000
# perBin = ((Range[1] - Range[0])/Bins)
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Signal$'
label2= r'$Background$'

labels=[label1,label2]
colors=["#1f77b4","#d62728"]

data = [
        DataFrames["Signal_Ds*"][i], # (DataFrames["Signal"]['Ds_charge']==-1) & 
        DataFrames["All_Ds*"][i]
       ]


plt.hist(data, color=colors, label=labels, alpha=1, range=Range, stacked=Stacked, density=Density, linewidth=2, bins=Bins, histtype='step')
# plt.axvspan(Range[0],0.115,color='gray',alpha=0.2)
# plt.axvline(0.115,ls='--',color='gray')
# plt.axvline(0.18,ls='--',color='gray')
# plt.axvspan(0.18,Range[1],color='gray',alpha=0.2)

# Title
#--------
# plt.title(r'$Truth-matching$', loc = "left")
# plt.title(r'$\bf Signal \; Events$', loc="right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{*+} - D_s^{+})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 32000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
Stacked = True
Density = False
Bins = 50
i = 'Ds_star_mass_diff_pi'
# i = 'Ds_massDifference_0'
Range = [0.0, 0.4]
Op = -1
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$\gamma \; from \; \pi^{0}$'
label2= r'$\gamma \; from \; \pi^{+}$'
label3= r'$Other$'
label4= r'$\gamma \; from \; D^{*0}$'

labels=[label1,label2,label3,label4]
colors = ["#1f77b4",  # blue
          "#2ca02c",  # green
          "#d62728",  # red
          "#9467bd"]  # purple
data=[
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["gamma_genMotherPDG"])==111)][i],
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["gamma_genMotherPDG"])==211)][i],
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["gamma_genMotherPDG"])!=111) & (abs(DataFrames["All_Ds*"]["gamma_genMotherPDG"])!=211) & (abs(DataFrames["All_Ds*"]["gamma_genMotherPDG"])!=423)][i],
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["gamma_genMotherPDG"])==423)][i],
      ]

# factor = 0.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
plt.title(r'$Truth-matching$', loc = "left")
plt.title(r'$Generic \; MC: \; \int\mathcal{L}dt\approx\;1.4$ ab$^{-1}$', loc="right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{*+} - D_s^{+})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 32000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
i = 'Ds_star_mass_diff_pi'
# i = 'Ds_massDifference_0'
Range = [0.0, 0.4]
Op = -1
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$23$'
label2= r'$N/A$'
label3= r'$D^{*0}$'
label4= r'$D^{*+}$'
label5= r'Other'

labels=[label1,label2,label3,label4,label5]
# colors = ["#1f77b4",  # blue
#           "#2ca02c",  # green
#           "#d62728",  # red
#           "#9467bd"]  # purple
data=[
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["Ds_c_mcPDG"])==23)][i],
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["Ds_c_mcPDG"]).isna())][i],
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["Ds_c_mcPDG"])==423)][i],
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["Ds_c_mcPDG"])==413)][i],
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["Ds_c_mcPDG"])!=23) & (~abs(DataFrames["All_Ds*"]["Ds_c_mcPDG"]).isna()) & (abs(DataFrames["All_Ds*"]["Ds_c_mcPDG"])!=423) & (abs(DataFrames["All_Ds*"]["Ds_c_mcPDG"])!=413)][i],
      ]

# factor = 0.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
plt.title(r'$Truth-matching$', loc = "left")
plt.title(r'$Inclusive \; MC: \; \int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc="right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{*+} - D_s^{+})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 32000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
print(DataFrames["All_Ds*"][['Ds_c_mcPDG']].value_counts(normalize=True,dropna=False).apply(lambda x: f"{x:.6f}"))

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'Ds_Ds_starminusDs_M_Correction'
# i = 'Ds_massDifference_0'
Range = [0.0, 0.4]
BS = 0.566
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$D^{0}$'
label3= r'$D^{*0} \rightarrow D^{0} \; \pi^{0} / \gamma$'
label4= r'$D^{*+} \rightarrow D^{0} \; \pi^{+}$'

labels=[label1,label2,label3,label4]
colors=['C5','C4','C1','C2',]
data=[
      DataFrames[Samples][((abs(DataFrames[Samples]['Ds_D0_Other'])==1) | ((abs(DataFrames[Samples]['D0_mcPDG'])==421) & (abs(DataFrames[Samples]['D0_isSignal'])==0)))][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]["Ds_BkgBDT"]>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]["Ds_BkgBDT"]>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=2, range=Range)
# plt.axvspan(Range[0],0.16,color='gray',alpha=0.2)
# plt.axvline(0.16,ls='--',color='gray')

# Title
#--------
plt.title(r'$\bf Generic \; Events$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 30000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
i = 'Ds_star_mass_diff_pi'
# i = 'Ds_massDifference_0'
Range = [0.0, 0.4]
Op = -1
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Comb.$'
label2= r'$NaN$'

labels=[label1,label2]
colors=["#DD8452","#C44E52"]
data=[
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["Ds_star_mcPDG"])==23) | (abs(DataFrames["All_Ds*"]["Ds_star_mcPDG"])==300553)][i],
      DataFrames["All_Ds*"][(DataFrames["All_Ds*"]["Ds_star_mcPDG"].isna())][i],
      ]

# factor = 0.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
plt.title(r'$Truth-matching$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc="right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{*+} - D_s^{+})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 80000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
i = 'Ds_star_mass_diff_pi'
# i = 'Ds_massDifference_0'
Range = [0.0, 0.4]
Op = -1
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Comb.$'
label2= r'$NaN$'

labels=[label1,label2]
colors=["#DD8452","#C44E52"]
data=[
      DataFrames["All_Ds*"][(abs(DataFrames["All_Ds*"]["Ds_star_mcPDG"])==23) | (abs(DataFrames["All_Ds*"]["Ds_star_mcPDG"])==300553)][i],
      DataFrames["All_Ds*"][(DataFrames["All_Ds*"]["Ds_star_mcPDG"].isna())][i],
      ]

# factor = 0.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
plt.title(r'$Truth-matching$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc="right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{*+} - D_s^{+})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 80000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
Range = [0, 0.4]
Hits = -1
Op = 0.58
RR = 0.4
BS = 0.5
FS = 0.5
xL = 0.12
xR = 0.16
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

plt.hist(DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames["Signal_Ds*"]['Ds_star_pi0vetoDsstar_M_Correction']<=xR) & (DataFrames["Signal_Ds*"]['Ds_star_pi0vetoDsstar_M_Correction']>=xL)]['Ds_star_massDifference_0'], label=r'$Signal \; Events$', density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.hist(DataFrames["All_Ds*"][(DataFrames["All_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["All_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["All_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["All_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames["All_Ds*"]['Ds_star_pi0vetoDsstar_M_Correction']<=xR) & (DataFrames["All_Ds*"]['Ds_star_pi0vetoDsstar_M_Correction']>=xL)]['Ds_star_massDifference_0'], label=r'$Generic \; Events$', density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.12,color='gray',alpha=0.2)
# plt.axvline(0.12,ls='--',color='gray')
# plt.axvspan(0.165,Range[1],color='gray',alpha=0.2)
# plt.axvline(0.165,ls='--',color='gray')

# Title
#---------
# plt.title(r'$nPXDHits(e^{+})>0$'+'\n'+r'$BDT \geq 0.58$', loc = "left")
plt.title(r'$\bf Generic\;Events$'+'\n'+r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# plt.title(r'$2M \; Events$', loc = "left")
# plt.title(r'$\bf Signal\;Events$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{*+} - D_s^{+})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.ylim(0, 18000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = True
Bins = 50
Range = [0.08, 0.2]
Hits = -1
Op = 0.58
RR = 0.4
BS = 0.5
FS = 0.5
var = 'gamma_clusterTiming'
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$isSignal(D_s^{*+})=1$'
label2= r'$isSignal(D_s^{*+})=0$'
label3= r'$NaN$'

labels=[label1,label2,label3]
colors=['#fd7f6f','#7eb0d5','purple']

data = [DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames[samples[1]]['Ds_star_isSignal']==1)][var],
        DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames[samples[1]]['Ds_star_isSignal']==0)][var],
        DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames[samples[1]]['Ds_star_isSignal'].isna())][var]
       ]


plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')

# Title
#---------
plt.title(r'$D_s^{*+} \rightarrow D_s^{+} \gamma_{sig}$', loc = "left")
plt.title(r'$\bf Signal\;Events$', loc = "right")
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$m(\gamma_{sig}\gamma_{ROE})\;[GeV/c^{2}]$')
# plt.yscale("log") 
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
Range = [0.0, 0.4]
Hits = 0
Op = 0.58
var = 'Ds_star_mass_diff_pi'
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$isSignal(D_s^{*+})=1$'
label2= r'$isSignal(D_s^{*+})=0$'
label3= r'$NaN$'

labels=[label1,label2,label3]
colors=['#fd7f6f','#7eb0d5','purple']

data = [DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal']==1][var],
        DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal']==0][var],
        DataFrames[samples[1]][DataFrames[samples[1]]['Ds_star_isSignal'].isna()][var]
       ]


plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')

# Title
#---------
# Signal
plt.title(r'$2M\;Events$', loc = "left")
plt.title(r'$\bf Signal\;Events$', loc = "right")
# # Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log") 
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
Range = [0, 1]
Op = 0.58
var = 'gamma_clusterZernikeMVA'
perBin = ((Range[1] - Range[0])/Bins)
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$mcPDG=22$'
label2= r'$mcPDG!=22$'
label3= r'$NaN$'

labels=[label1,label2,label3]
colors=['#fd7f6f','#7eb0d5','purple']

data = [abs(DataFrames[samples[1]][(DataFrames[samples[1]]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames[samples[1]]['gamma_mcPDG']==22)][var]),
        abs(DataFrames[samples[1]][(DataFrames[samples[1]]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames[samples[1]]['gamma_mcPDG']!=22)][var]),
        abs(DataFrames[samples[1]][(DataFrames[samples[1]]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames[samples[1]]['gamma_isSignal'].isna())][var])
       ]


plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')

# Title
#---------
# plt.title(r'$BDT \geq 0.58$', loc = "left")
# plt.title(r'$2M\;Events$', loc = "left")
plt.title(r'$\bf Signal\;Events$', loc = "right")
# # Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f})$'.format(width = perBin))
plt.xlabel(r'$clusterZernikeMVA \; (\gamma)$')
# plt.yscale("log") 
# plt.legend(loc='upper center')
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = True
Bins = 50
Range = [0, 1]
Op = 0.58
var = 'gamma_clusterE1E9'
perBin = ((Range[1] - Range[0])/Bins)
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$isSignal(\gamma)=1$'
label2= r'$isSignal(\gamma)=0$'
label3= r'$NaN$'

labels=[label1,label2,label3]
colors=['#fd7f6f','#7eb0d5','purple']

data = [abs(DataFrames[samples[1]][(DataFrames[samples[1]]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames[samples[1]]['gamma_isSignal']==1)][var]),
        abs(DataFrames[samples[1]][(DataFrames[samples[1]]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames[samples[1]]['gamma_isSignal']==0)][var]),
        abs(DataFrames[samples[1]][(DataFrames[samples[1]]['Ds_c_extraInfo_FastBDT']>=Op) & (DataFrames[samples[1]]['gamma_isSignal'].isna())][var])
       ]


plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')
plt.axvspan(Range[0],0.5,color='gray',alpha=0.2)
plt.axvline(0.5,ls='--',color='gray')

# Title
#---------
# plt.title(r'$BDT \geq 0.58$', loc = "left")
# plt.title(r'$2M\;Events$', loc = "left")
plt.title(r'$\bf Signal\;Events$', loc = "right")
# # Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f})$'.format(width = perBin))
plt.xlabel(r'$fakePhotonSuppressionScore \; (\gamma)$')
# plt.yscale("log") 
plt.legend(loc='upper center')
# plt.legend()
plt.show()

# Two Variable

In [ ]:
FS = 0.5
BS = 0.5

# Generate random data 
x = DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_mcPDG']==22) & (DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>0.4) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['gamma_clusterZernikeMVA'] 
y = DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_mcPDG']==22) & (DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>0.4) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['gamma_minC2TDist']
  
# Create a 2-D histogram 
plt.hist2d(x, y, 
           bins=(50, 50), range=[(0,1),(0,200)],
           cmap='viridis',
           norm=LogNorm()) 
plt.colorbar() 
  
# Add labels and title 
plt.xlabel(r'$clusterZernikeMVA \; (\gamma)$') 
plt.ylabel(r'$minC2TDist \; (\gamma) \; [cm]$') 
plt.title(r'$mcPDG=22$') 
  
# Display the plot 
plt.show()

In [ ]:
# Generate random data 
x = DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_mcPDG']!=22) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['gamma_clusterZernikeMVA'] 
y = DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_mcPDG']!=22) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['gamma_minC2TDist']
  
# Create a 2-D histogram 
plt.hist2d(x, y, 
           bins=(50, 50), range=[(0,1),(0,200)],
           cmap='viridis',
           norm=LogNorm()) 
plt.colorbar() 
  
# Add labels and title 
plt.xlabel(r'$clusterZernikeMVA \; (\gamma)$') 
plt.ylabel(r'$minC2TDist \; (\gamma) \; [cm]$') 
plt.title(r'$mcPDG!=22$') 
  
# Display the plot 
plt.show()

In [ ]:
# Generate random data 
x = DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_mcPDG']!=22) & (DataFrames["Signal_Ds*"]['gamma_Region3']==1) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['gamma_clusterZernikeMVA'] 
y = DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_mcPDG']!=22) & (DataFrames["Signal_Ds*"]['gamma_Region3']==1) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['gamma_minC2TDist']
  
# Create a 2-D histogram 
plt.hist2d(x, y, 
           bins=(50, 50), range=[(0,1),(0,200)],
           cmap='viridis',
           norm=LogNorm()) 
plt.colorbar() 
  
# Add labels and title 
plt.xlabel(r'$clusterZernikeMVA \; (\gamma)$') 
plt.ylabel(r'$minC2TDist \; (\gamma) \; [cm]$') 
plt.title(r'$mcPDG!=22(E \geq 500 \; MeV)$')
  
# Display the plot 
plt.show()

In [ ]:
DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)][['gamma_clusterZernikeMVA','gamma_minC2TDist']].corr()

# Photon MVA

In [ ]:
from Functions import get_df_sig_bkg,get_nsig_nbkg,plot_var_Real_Fake,optimize_RF,plot_save

optimize_RF(df_sig=DataFrames[samples[1]][DataFrames[samples[1]]['gamma_isSignal']==1],
            df_bkg=DataFrames[samples[1]][(DataFrames[samples[1]]['gamma_isSignal']==0) | (DataFrames[samples[1]]['gamma_isSignal'].isna())],
            var='gamma_fakePhotonSuppressionScore',
            Signal=DataFrames[samples[1]][DataFrames[samples[1]]['gamma_isSignal']==1],
            Background=DataFrames[samples[1]][(DataFrames[samples[1]]['gamma_isSignal']==0) | (DataFrames[samples[1]]['gamma_isSignal'].isna())],
            FoM='gamma_fakePhotonSuppressionScore',
            xlabel='fakePhotonSuppression',
            Bins=50,Range=[0,1],
            varmin=0,varmax=0.98,select='right',Width=False)

# Catergory Split

In [ ]:
Stacked = False
Density = True
Bins = 50
i = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.8]
Cat = 0
Op = 0.58
dM = 0.1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

RE = (len(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op) & (DataFrames["Signal"]["Ds_goodDsplus"]==Cat)])/(2000000))*100

label1= r'$other$'
label2= r'$Comb.$'
label3= r'$Comb. Ds$'
label4= r'$D^{*0}$'
label5= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'

labels=[label1,label2,label3,label4,label5]
colors=['C2','C1','C3','C4','C5']
data=[
      DataFrames["All"][(abs(DataFrames["All"]["Ds_Other"])==1)   & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op) & (DataFrames["All"]["Ds_goodDsplus"]==Cat)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_mcPDG"])==23)  & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op) & (DataFrames["All"]["Ds_goodDsplus"]==Cat)][i],
      DataFrames["All"][(DataFrames["All"]["Ds_mcPDG"].isna())    & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op) & (DataFrames["All"]["Ds_goodDsplus"]==Cat)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_mcPDG"])==423) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op) & (DataFrames["All"]["Ds_goodDsplus"]==Cat)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_mcPDG"])==413) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op) & (DataFrames["All"]["Ds_goodDsplus"]==Cat)][i]
      ]

factor = 0.1
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op) & (DataFrames["Signal"]["Ds_goodDsplus"]==Cat)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op) & (DataFrames["Signal"]["Ds_goodDsplus"]==Cat)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
# plt.title(r'$nPXDHits(e^{+})>0$', loc = "left")
# plt.title(r'$BDT \geq 0.58$', loc = "left")
plt.title(r'$nPXDHits(e^{+})>0$'+'\n'+r'$BDT \geq 0.58$', loc = "left")
# plt.title('Reconstruction Efficiency:{eff:.2f}%'.format(eff = RE), loc = "Right")
plt.title('Category:2', loc = "Right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
plt.ylim(0, 12)
plt.legend()
plt.show()

# Categories

In [ ]:
Norm = True
Hits = 0
Op = 0.58
print(DataFrames["Signal"][(DataFrames["Signal"]['e_nPXDHits']>Hits) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op)]["Ds_goodDsplus"].value_counts(normalize=Norm,dropna=False))
print(DataFrames["All"][(DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)]["Ds_goodDsplus"].value_counts(normalize=Norm,dropna=False))

# Truth

In [ ]:
Op = 0.58
RR = -1
BS = -1
FS = -1
print(abs(DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['Ds_star_mcPDG']).value_counts(dropna=False))
print(abs(DataFrames["All_Ds*"][(DataFrames["All_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["All_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["All_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["All_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['Ds_star_mcPDG']).value_counts(dropna=False))

In [ ]:
len(DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)])

In [ ]:
len(DataFrames["All_Ds*"][(DataFrames["All_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["All_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["All_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["All_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)])

In [ ]:
print(DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['gamma_mcPDG'].value_counts(dropna=False))
print(DataFrames["All_Ds*"]['gamma_mcPDG'].value_counts(dropna=False))

In [ ]:
print(DataFrames["Signal_Ds*"][(DataFrames["Signal_Ds*"]['gamma_clusterE1E9']>=RR) & (DataFrames["Signal_Ds*"]['gamma_beamBackgroundSuppressionScore']>=BS) & (DataFrames["Signal_Ds*"]['gamma_fakePhotonSuppressionScore']>=FS) & (DataFrames["Signal_Ds*"]['Ds_c_extraInfo_FastBDT']>=Op)]['gamma_genMotherPDG'].value_counts(normalize=True,dropna=False))
print(DataFrames["All_Ds*"]['gamma_genMotherPDG'].value_counts(dropna=False))

In [ ]:
print(len(DataFrames["Signal_Ds*"]))
print(len(DataFrames["All_Ds*"]))